# Classifying Chargeback: a Data Science Capstone Project

This notebook details the design of a ML classification model which helped Credorax, a smart payments provider, tackle one of the biggest nuisances to the business: chargeback. After exploring and cleaning the data provided, our team (shoutout to Amy, Adi, and Vladi!) managed to design a model which, if implemented, would **increase profits by an average of 2%.**

My personal contributions to the projects centered on feature engineering, translating the business problem into appropriate evaluation metrics, and optimizing some of the classification models. With my peers' permission, I've taken the liberty of organizing the entire project in a single notebook (as opposed to the multiple Google Colab files we worked with) in order to help aspiring data scientist understand good practices and techniques when analyzing real-world data for real-world business problems.

So, without further ado, let's tackle a real-world business problem with Machine Learning! I hope you enjoy the ride.

## Contents

table of contents goes here, with links

## The Business Problem

**Chargeback** is the payment amount that is returned to a debit or credit card, after a customer disputes the transaction or simply returns the purchased item. The chargeback process can be initiated by either the merchant or the cardholder’s issuing bank. Merchants typically incur a significant fee from the card issuer when a chargeback occurs. 

Since part of Credorax's business is enabling transactions for merchants (and taking a percentage of the transaction), they are negatively affected when they fail to preemptively block a sale in which the chargeback ultimately occurs: this false negative (type II) error results in loss of business due to an unsatisfied merchant using Credorax's services. However, Credorax naturally can't afford to label valid transactions as chargebacks (a false positive, type I error) - aside from the loss of business from the sale, Credorax's reputation is tarnished and the customer's lifetime value will decrease. Let's summarize these scenarios in a table, along with the company's estimates of profits or losses for each case:

| &darr; Prediction \ Actual &rarr;| No Chargeback | Chargeback |
| :-: | :-: | :-: |
| **No Chargeback** | **TN** - legitimate transaction authorized, company **earns 15% of transaction amount** | **FN** - fradulant transaction authorized, company **loses 230% of transaction amount** (e.g. merchant compensation)|
| **Chargeback** | **FP** - legitimate transaction blocked, company **loses 200% of transaction amount** (due to angry merchants and customers) | **TP** - fradulant transaction blocked, **no profit or loss**|

It is therefore quite clear how an appropriate **classification model** can improve Credorax's profitability, and for that purpose our team  was tasked with designing a model which would improve the company's baseline strategy of labeling all transactions as valid. But before we start examining the wide range of available models at our disposal, we must fully understand the data we're given and determine how it can best be used for our models.

So - what must a prudent data scientist do before modeling? Here's a simple checklist to get us started:

1. **Loading the data.** This may be trivial if we're given a single CSV or JSON file, but in many cases we'll be given our data in parts (and therefore will need to use **joins** and **concatenations**) or in more advanced formats (e.g. zipped or pickled files).
 
2. **Examining the data.** What is the **size** of our dataset? How many **features** do we have to work with, and are they all relevant? Are there many **missing values** for some of the features? Are the features **correlated**?   In the vast majority of cases, the answers to these questions will involve:
 
3. **Visualizing the data**, by using Python's major packages, such as **pandas** for data structure and **matplotlib** or **seaborn** for basic and advanced charts and plots (but there are many options, choose what suits you best!). Common visualizations include bar charts for examining the distribution of categorical data, scatter plots and heat-maps to examine correlation, or perhaps even a violin plot (good for comparing two groups side by side) if you're feeling fancy. 
 
4. **Transforming the data**. In some cases, we'll need to make some changes before we're ready to unleash some models: we might need to **impute** missing data (perhaps by using the mean) or **drop** the problematic features or samples entirely; we may choose to **disregard features due to irrelevancy or high correlation** to other features 

The aforementioned are often summarized as **ETL** ([Extract, Transform, Load](https://en.wikipedia.org/wiki/Extract,_transform,_load)), which is a three-phase process where data is extracted, transformed (cleaned, sanitized, scrubbed) and loaded into an output data container, and **EDA** ([Exploratory Data Analysis](https://en.wikipedia.org/wiki/Exploratory_data_analysis)), a highly visual and methodical approach of analyzing datasets. Further reading about both topics is encouraged!

It should go without saying that this list is not exhaustive, and you'll need to think long and hard about the dataset you're working with and how to best use it. In addition, data science is an **iterative** process: you'll be repeating the steps above many times as you get to know your data better, and once you've run a basic model or two you'll think of many other ways the data must be examined or transformed in order to get better results. You might even realize you need more data! Here's the CRISP-DM (short for [CRoss-Industry Standard Process for Data Mining](https://en.wikipedia.org/wiki/Cross-industry_standard_process_for_data_mining)) methodology which was presented in our class and highlights this point:

<img src="CRISP-DM_Process_Diagram.png" alt="Drawing" style="width: 400px;"/>

(Image by Kenneth Jensen - Own work based on:ftp://public.dhe.ibm.com/software/analytics/spss/documentation/modeler/18.0/en/ModelerCRISPDM.pdf (Figure 1), CC BY-SA 3.0, https://commons.wikimedia.org/w/index.php?curid=24930610)

Let's dive into our data and see what we have to work with.

## Data Overview

The data provided by the company can be split into two categories: data on the *transaction* level (millions of entries loaded and concatenated from zipped files), and *general* data (provided in separate CSV tables) about the merchants, credit card security measures, location and more. The tranasctional data contains keys which are linked to the general data tables, allowing us to use joins to create one large dataset which will ultimately be used for the classification models. The data can be summarized as follows:

| Table | Format | # Files | Size | Description |
| :-: | :-: | :-: | :-: |:-- |
| **DIM_AVS_RESPONSE** | CSV | 1 | (25, 7) | The type of response the AVS credit card security measure returns: match or mismatch for address and postal code. |
| **DIM_CVV_RESPONSE** | CSV | 1 | (8, 4) | The response for the CVV (3/4 numbers on back of card - did the buyer list them properly. |
| **DIM_DISPUTE_REASON** | CSV | 1 | (328, 4) | Detailed explanation of the dispute reason which led to the chargeback. |
| **DIM_MCC** | CSV | 1 | (1353,5) | Merchant details, including if industry and sector are high-risk (think online gambling) |
| **DIM_TRX_REGION** | CSV | 1 | (129310, 23) | Data relating to regions and location, e.g. billing address, IP location, merchant location. Also include credit card type (e.g. VISA / Mastercard).|
| **result_xx-05-2020** | CSV (zipped) | 31 | (6023968, 14) | The **main table** with all transactional data *except* the chargeback/no-chargeback labeling. |
| **labels_xx-xx-05-2020** | CSV (zipped) | 2 | (6038459, 2) | For every transaction, labels stating the chargeback reason index (which can simply be "no chargeback").|


We will now examine the data in greater depth. This will be done in two parts:

1. Analysis of the general data tables, in order to provide insight about the data and to determine which features would be useful for the classification model as-is, which ones should be dropped, and if there are new features which can be engineered based on the data.
2. Once the initial analysis has been conducted and the ETL process been refined accordingly, we load the full table of transaction data (and join the chargeback reason label and all other tables) and conduct further EDA analysis which will help us understand the relationship between the features and the target variable.

## ETL: Loading the Data and Creating Our Master Table

### Imports

Let's first import an array of useful Python packages which will assist us: 

In [1]:
import io
import matplotlib
import numpy as np
import pandas as pd
from os import path
from PIL import Image
import matplotlib.pyplot as plt

### AVS response analysis

The Address Verification Service (AVS) is a tool that enables merchants to detect suspicious credit card transactions and prevent credit card fraud. AVS verifies that the billing address entered by the customer is the same as the one associated with the cardholder’s credit card account. AVS response codes are returned to the merchant during the authorization process and help the merchant determine the next action, which could be transaction approval, exception, or decline. (from [Address Verification Service, Investopedia](https://www.investopedia.com/terms/a/address-verification-system.asp))

Let's take a look at the AVS response table the company provided.

In [2]:
avs_df = pd.read_csv('data/DIM_AVS_RESPONSE.csv')
avs_df.head()

,AVS_RESPONSE_KEY,ETL_RUN_DATE,ETL_CREATION_DATE,ETL_UPDATE_DATE,ETL_LOAD_NO,ETL_PROCESS_NAME,ETL_SOURCE,SOURCE_SYSTEM_KEY,AVS_RESPONSE_ID,AVS_RESPONSE_DESC
0,21,2017-12-02,2017-12-03,2021-03-02,280,m_stg_dim_avs_response,GLOBAL,14,E,AVS service is not available for the particula...
1,22,2017-12-02,2017-12-03,2021-03-02,280,m_stg_dim_avs_response,GLOBAL,14,-,AVS service is not available for the particula...
2,-99,2014-04-17,2014-04-17,2014-04-17,0,Manual,GLOBAL,15,-1,Unknown AVS Response
3,155,2017-12-02,2014-05-19,2021-03-02,1353,m_stg_dim_avs_response,GLOBAL,15,A,Partial Match - Address match; Zip/Postal Code...
4,156,2017-12-02,2014-05-19,2021-03-02,1353,m_stg_dim_avs_response,GLOBAL,15,B,Partial Match - Address match; Zip/Postal Code...


In [3]:
avs_df.shape

(25, 10)

Let's see how many distinct AVS categories there are:

In [4]:
avs_df[["AVS_RESPONSE_ID", "AVS_RESPONSE_DESC"]]

,AVS_RESPONSE_ID,AVS_RESPONSE_DESC
0,E,AVS service is not available for the particula...
1,-,AVS service is not available for the particula...
2,-1,Unknown AVS Response
3,A,Partial Match - Address match; Zip/Postal Code...
4,B,Partial Match - Address match; Zip/Postal Code...
5,Cod,No Match  Address and Zip/Postal Code not ver...
6,D,Full Match - Address and Zip/Postal Code match
7,F,Full Match - Address and Zip/Postal Code match...
8,G,No Match - Address not verified
9,I,Address not verified


We see that the AVS_RESPONSE_DESC feature contains text, which we see can be translated into two categorical variables of importance: whether or not the address matches, and whether or not the zipcode matches. (To keep things simpler, we're treating partial matches as matches for the time being). In both cases, the answer can be yes (1), no (-1), or unknown (0). We examine each description one-by-one in order to translate the AVS_RESPONSE_ID into these new features, and drop the remaining, unnecessary columns: 

In [5]:
avs_df['AVS_ADDRESS_MATCH'] = avs_df.AVS_RESPONSE_ID.apply(lambda x: 1 if x in ["D", "F", "X", "Y", "M", "A", "B"] 
                                                          else (-1 if x in ["P", "W", "Z", "N", "Cod", "C"] else 0))
# TODO: split this?
avs_df['AVS_ZIP_YES_MATCH'] = avs_df.AVS_RESPONSE_ID.apply(lambda x: 1 if x in ["D", "F", "X", "Y", "M", "P", "W", "Z"] else 0)
avs_df['AVS_ZIP_NO_MATCH'] = avs_df.AVS_RESPONSE_ID.apply(lambda x: 1 if x in ["A", "N"] else 0)

avs_df.drop(["ETL_RUN_DATE", "ETL_CREATION_DATE","ETL_UPDATE_DATE", "ETL_LOAD_NO", "ETL_PROCESS_NAME", "ETL_SOURCE", "SOURCE_SYSTEM_KEY"],
            inplace=True, axis=1) # Database variables, can be removed in all tables.

avs_df.head()

,AVS_RESPONSE_KEY,AVS_RESPONSE_ID,AVS_RESPONSE_DESC,AVS_ADDRESS_MATCH,AVS_ZIP_YES_MATCH,AVS_ZIP_NO_MATCH
0,21,E,AVS service is not available for the particula...,0,0,0
1,22,-,AVS service is not available for the particula...,0,0,0
2,-99,-1,Unknown AVS Response,0,0,0
3,155,A,Partial Match - Address match; Zip/Postal Code...,1,0,1
4,156,B,Partial Match - Address match; Zip/Postal Code...,1,0,0


These new categorical features are suitable for classification models, and we'll be making use of them later. Onwards to the CVV security check: 

### CVV response analysis

The CVV Number ("Card Verification Value") should be more familiar to you: it's the 3/4 digit number on your credit card or debit card which proves that you actually have the physical credit or debit card - and helps to keep you safe while reducing fraud. Let's look into the CVV data:

In [6]:
cvv_df = pd.read_csv('data/DIM_CVV_RESPONSE.csv')
cvv_df.drop(["ETL_RUN_DATE", "ETL_CREATION_DATE","ETL_UPDATE_DATE", "ETL_LOAD_NO", "ETL_PROCESS_NAME", "ETL_SOURCE",
             "SOURCE_SYSTEM_KEY"], inplace=True, axis=1) # Database variables, can be removed in all tables.
cvv_df

,CVV_RESPONSE_KEY,CVV_RESPONSE_ID,CVV_RESPONSE_DESC
0,-99,-1,Unknown CVV Response
1,119,M,CVV2/CVC2 Match
2,120,N,CVV2/CVC2 No Match
3,121,P,Not processed
4,122,S,The CVV2 should be on the card but the merchan...
5,123,U,CVV2/CVC2 Unavailable issuer does not support
6,124,Y,CVC1 Incorrect
7,-98,-1,Unknown CVV Response


Here too we're required to convert the description into features we can work with: CVV_YES_MATCH to indicate that the CVV matches <<>> (which may strongly indicate that the transaction is legitimate) and CVV_NO_MATCH if the reverse is true.

In [7]:
cvv_df['CVV_YES_MATCH'] = cvv_df.CVV_RESPONSE_ID.apply(lambda x: 1 if x == "M" else 0)
cvv_df['CVV_NO_MATCH'] = cvv_df.CVV_RESPONSE_ID.apply(lambda x: 1 if x in ["Y", "N"] else 0)
cvv_df

,CVV_RESPONSE_KEY,CVV_RESPONSE_ID,CVV_RESPONSE_DESC,CVV_YES_MATCH,CVV_NO_MATCH
0,-99,-1,Unknown CVV Response,0,0
1,119,M,CVV2/CVC2 Match,1,0
2,120,N,CVV2/CVC2 No Match,0,1
3,121,P,Not processed,0,0
4,122,S,The CVV2 should be on the card but the merchan...,0,0
5,123,U,CVV2/CVC2 Unavailable issuer does not support,0,0
6,124,Y,CVC1 Incorrect,0,1
7,-98,-1,Unknown CVV Response,0,0


### Merchant Information (MCC)

We usually think of chargeback as a problem in which merchants are the victim of unreliable (or malicious) customers. However, when you're a company whose profitability depends on making sure as many good transactions are going through and the bad ones are blocked in advnaced, it might be far more beneficial to put your focus on the stores themselves: perhaps the merchant is untrustworthy and there are many cases of legitimate sale cancellations (we don't want to work with those types), or alternatively the business sector might be one which is a hotbed for fraud and returns (online retail or gift cards, perhaps?).

Let's take a look at the data the company provides regarding the merchants who've accepted the credit cards in their store:

In [8]:
mcc_df = pd.read_csv('data/DIM_MCC.csv')
mcc_df.head()

,MCC_KEY,ETL_RUN_DATE,ETL_CREATION_DATE,ETL_UPDATE_DATE,ETL_LOAD_NO,ETL_PROCESS_NAME,ETL_SOURCE,ETL_SOURCE_PRIORITY,SOURCE_SYSTEM_KEY,MCC_ID,MCC_DESC,IS_VISA_HIGH_RISK,IS_MC_HIGH_RISK,MCC_GROUP
0,369,2015-07-26,2015-07-27,2018-12-17,659.0,m_stg_dim_mcc,GLOBAL,1,8,7395,Photo Finishing Laboratories and Photo Developing,0,0,Business & Professional Services
1,371,2015-07-26,2015-07-27,2018-12-17,659.0,m_stg_dim_mcc,GLOBAL,1,8,7512,Automobile Rental Agency,0,0,Vehicle Hire
2,372,2015-07-26,2015-07-27,2018-12-17,659.0,m_stg_dim_mcc,GLOBAL,1,8,7519,Motor Home and Recreational Vehicle Rentals,0,0,Vehicle Hire
3,373,2015-07-26,2015-07-27,2018-12-17,659.0,m_stg_dim_mcc,GLOBAL,1,8,7523,Parking Lots and Garages,0,0,Motoring
4,293,2015-07-26,2015-07-27,2018-12-17,659.0,m_stg_dim_mcc,GLOBAL,1,8,5722,Household Appliance Stores,0,0,Wholesale


In [9]:
mcc_df.shape

(1353, 14)

Aside from the IS_MC_HIGH_RISK and IS_MC_HIGH_RISK features which are likely to be useful, we note two categorical variables: MCC_GROUP which denotes the general category of service the merchant provides, and MCC_DESC which goes into further detail. Since these categorical features must be one-hot encoded later, let's see how many unique values there are for each feature:

In [10]:
mcc_df['MCC_GROUP'].nunique()

26

In [11]:
mcc_df['MCC_DESC'].nunique()

927

While the detailed Merchant Description category might be helpful (e.g. within the Entertainment sector, a business focusing on gambling might have a higher risk than a movie theater), the sheer quantity of subgroups would require lots of computational time and increase the risk for overfitting. We'll keep things simple for now and leave this feature out, while keeping the MCC_GROUP feature; we can always re-examine this assumption later.

After identifying the features of interest, the remaining features (which mainly relate to when the merchant info has been added to the database) can be removed.

In [12]:
mcc_df.drop(["ETL_RUN_DATE", "ETL_CREATION_DATE","ETL_UPDATE_DATE", "ETL_LOAD_NO", "ETL_PROCESS_NAME", "ETL_SOURCE",
            "ETL_SOURCE_PRIORITY", "SOURCE_SYSTEM_KEY"], inplace=True, axis=1) # Database variables

mcc_df.drop("MCC_DESC", inplace=True, axis=1)
mcc_df

,MCC_KEY,MCC_ID,IS_VISA_HIGH_RISK,IS_MC_HIGH_RISK,MCC_GROUP
0,369,7395,0,0,Business & Professional Services
1,371,7512,0,0,Vehicle Hire
2,372,7519,0,0,Vehicle Hire
3,373,7523,0,0,Motoring
4,293,5722,0,0,Wholesale
...,...,...,...,...,...
1348,3007,7280,0,0,Unknown MCC Group
1349,3008,7295,0,0,Unknown MCC Group
1350,3009,8912,0,0,Unknown MCC Group
1351,3010,9400,0,0,Unknown MCC Group


A quick check tells us that there are 6 high-risk Visa categories and 11 high-risk MC categories, we'll see later if those are a strong chargeback indicator.

In [13]:
mcc_df[["IS_VISA_HIGH_RISK", "IS_MC_HIGH_RISK"]].sum(axis=0)

IS_VISA_HIGH_RISK     6
IS_MC_HIGH_RISK      11
dtype: int64

### Regions

Regional data can also be extremely important in preemptive chargeback identification. Some third-world countries have a negative reputation in manners relating to fraud, and international trade may exhibit different behavior than local sales. 

Let's look at the data:

In [14]:
region_df = pd.read_csv('data/DIM_TRX_REGION.csv')
region_df.head()

,TRX_REGION_KEY,ETL_RUN_DATE,ETL_CREATION_DATE,ETL_UPDATE_DATE,ETL_LOAD_NO,ETL_PROCESS_NAME,ETL_SOURCE,SOURCE_SYSTEM_KEY,EFFECTIVE_START_DATE,EFFECTIVE_END_DATE,...,BIN_COUNTRY_DESC,MERCHANT_COUNTRY_ALPHA_3_ID,MERCHANT_COUNTRY_ALPHA_2_ID,MERCHANT_COUNTRY_NUMERIC_ID,MERCHANT_COUNTRY_DESC,CARD_SCHEME_ID,CARD_SCHEME_TRX_REGION,CRX_TRX_REGION,MD5_KEY,ETL_SOURCE_PRIORITY
0,2833386,2017-04-01,2017-07-27,2017-07-27,51.0,m_stg_dim_trx_region,GLOBAL,7.0,2017-04-01,2999-12-31,...,Cyprus,RUS,RU,643,Russian Federation,Visa,Interregional,Inter,456219565EDF917386F5EABFBEE9D09C,NaN
1,2833387,2017-04-01,2017-07-27,2017-07-27,51.0,m_stg_dim_trx_region,GLOBAL,7.0,2017-04-01,2999-12-31,...,Czech Republic,FRA,FR,250,France,Mastercard,Intra -Sepa,Intra,4761CD6E93225E6D0AB9E55FBBA037DD,NaN
2,2833388,2017-04-01,2017-07-27,2017-07-27,51.0,m_stg_dim_trx_region,GLOBAL,7.0,2017-04-01,2999-12-31,...,Czech Republic,PHL,PH,608,Philippines,Visa,Interregional,Inter,456219565EDF917386F5EABFBEE9D09C,NaN
3,2833389,2017-04-01,2017-07-27,2017-07-27,51.0,m_stg_dim_trx_region,GLOBAL,7.0,2017-04-01,2999-12-31,...,Czech Republic,ROU,RO,642,Romania,Visa,Intraregional,Intra,9CA5718096E8BD793BAA1A3E87DF1B34,NaN
4,2833390,2017-04-01,2017-07-27,2017-07-27,51.0,m_stg_dim_trx_region,GLOBAL,7.0,2017-04-01,2999-12-31,...,Czech Republic,SGS,GS,239,South Georgia and the South Sandwich Islands,Visa,Intraregional,Intra,9CA5718096E8BD793BAA1A3E87DF1B34,NaN


In [15]:
region_df.shape

(129310, 23)

Aside from lots of features relating to countries - some of which seem to be redundant - we also note that the Card Scheme ID (e.g. Visa or Mastercard) is listed here too. Let's clean the unneccesary database variables, and we'll take care of the rest later. 

In [16]:
region_df.drop(["ETL_RUN_DATE", "ETL_CREATION_DATE","ETL_UPDATE_DATE", "ETL_LOAD_NO", "ETL_PROCESS_NAME", "ETL_SOURCE",
            "SOURCE_SYSTEM_KEY", "EFFECTIVE_START_DATE","EFFECTIVE_END_DATE"], inplace=True, axis=1) # Database variables

In [17]:
region_df.head()

,TRX_REGION_KEY,BIN_COUNTRY_ALPHA_3_ID,BIN_COUNTRY_ALPHA_2_ID,BIN_COUNTRY_NUMERIC_ID,BIN_COUNTRY_DESC,MERCHANT_COUNTRY_ALPHA_3_ID,MERCHANT_COUNTRY_ALPHA_2_ID,MERCHANT_COUNTRY_NUMERIC_ID,MERCHANT_COUNTRY_DESC,CARD_SCHEME_ID,CARD_SCHEME_TRX_REGION,CRX_TRX_REGION,MD5_KEY,ETL_SOURCE_PRIORITY
0,2833386,CYP,CY,196,Cyprus,RUS,RU,643,Russian Federation,Visa,Interregional,Inter,456219565EDF917386F5EABFBEE9D09C,NaN
1,2833387,CZE,CZ,203,Czech Republic,FRA,FR,250,France,Mastercard,Intra -Sepa,Intra,4761CD6E93225E6D0AB9E55FBBA037DD,NaN
2,2833388,CZE,CZ,203,Czech Republic,PHL,PH,608,Philippines,Visa,Interregional,Inter,456219565EDF917386F5EABFBEE9D09C,NaN
3,2833389,CZE,CZ,203,Czech Republic,ROU,RO,642,Romania,Visa,Intraregional,Intra,9CA5718096E8BD793BAA1A3E87DF1B34,NaN
4,2833390,CZE,CZ,203,Czech Republic,SGS,GS,239,South Georgia and the South Sandwich Islands,Visa,Intraregional,Intra,9CA5718096E8BD793BAA1A3E87DF1B34,NaN


### Dispute Reason

This is the fun part: all of the various possible excuses which can be given for chargeback. Let's first load and examine the data:

In [18]:
dispute_reason_df = pd.read_csv('data/DIM_DISPUTE_REASON.csv')
dispute_reason_df.head()

,DISPUTE_REASON_KEY,ETL_RUN_DATE,ETL_CREATION_DATE,ETL_UPDATE_DATE,ETL_LOAD_NO,ETL_PROCESS_NAME,ETL_SOURCE,ETL_SOURCE_PRIORITY,SOURCE_SYSTEM_KEY,CARD_SCHEME_ID,CARD_SCHEME_DESC,DISPUTE_REASON_CODE,DISPUTE_REASON_BUSINESS_CODE,DISPUTE_REASON_DESC,DISPUTE_GROUP_DESC,WORKFLOW_TYPE
0,126,2019-07-28,2019-07-29,2021-03-02,883,m_stg_dim_dispute_reason,WAY4_PRE,1,2,X,Amex,4799,4799,Fraud Liability Shift - Lost or Stolen,Unknown Dispute Group,Unknown Workflow Type
1,127,2019-07-28,2019-07-29,2021-03-02,883,m_stg_dim_dispute_reason,WAY4_PRE,1,2,S,Visa,VC41,41,Cancelled recurring transaction,Old Visa Chargeback Reason Code,Old Visa Chargeback Reason Code
2,128,2019-07-28,2019-07-29,2021-03-02,883,m_stg_dim_dispute_reason,WAY4_PRE,1,2,S,Visa,VC77,77,Non-matching account number,Old Visa Chargeback Reason Code,Old Visa Chargeback Reason Code
3,129,2019-07-28,2019-07-29,2021-03-02,883,m_stg_dim_dispute_reason,WAY4_PRE,1,2,S,Visa,VC86,86,Paid by other means (U.S. only),Old Visa Chargeback Reason Code,Old Visa Chargeback Reason Code
4,130,2019-07-28,2019-07-29,2021-03-02,883,m_stg_dim_dispute_reason,WAY4_PRE,1,2,X,Amex,4517,4517,Request for Support Illegible/Incomplete,Unknown Dispute Group,Unknown Workflow Type


In [19]:
dispute_reason_df.shape

(330, 16)

Let's drop the columns we know are irrelevant and closely examine the Dispute Reason Key, since (as we'll see in the next section) this is our **target variable**.

In [20]:
dispute_reason_df.drop(["ETL_RUN_DATE", "ETL_CREATION_DATE","ETL_UPDATE_DATE", "ETL_LOAD_NO", "ETL_PROCESS_NAME", "ETL_SOURCE",
            "ETL_SOURCE_PRIORITY", "SOURCE_SYSTEM_KEY"], inplace=True, axis=1) # Database variables
dispute_reason_df.drop(["CARD_SCHEME_DESC", "DISPUTE_REASON_CODE", "CARD_SCHEME_ID"], inplace=True, axis=1) 
# The above variables appear in the main transaction database, therefore redundant
dispute_reason_df.drop(["DISPUTE_REASON_BUSINESS_CODE"], inplace=True, axis=1) # Unnecessary

In [21]:
dispute_reason_df[['DISPUTE_REASON_KEY', 'DISPUTE_REASON_DESC']].sample(15)

,DISPUTE_REASON_KEY,DISPUTE_REASON_DESC
256,246,Late presentment
303,293,Fraud. Trans. Exceeds Fl. Limit and not Author...
113,31,VCR Duplicate Processing/Paid by Other Means
115,33,VCR Processing Errors Category
173,91,For future chip card initiatives
8,134,Unauthorized Purchase
265,255,Cardholder does not recognise transaction
51,177,Counterfeit Trans. Magn. Stripe POS Fraud
213,322,Legal process request
95,13,Declined authorization


Note that the reasons for chargeback substantially differ from one another. If we receive an additional transaction from a vendor for whom a previous transaction was labeled "fraud investigation", that'd raise a red flag, whereas "missing signature" can indicate technical difficulties which we expect to get resolved later, meaning that the risk for future transaction rejection is low.

This observation has serious ramifications on our classification strategy. Do we want our model to simply learn the binary yes/no chargeback variable, or try to distinguish between the different chargeback types in an attempt to refine the model?

We'll continue the discussion in the next section, but in the meantime it's worth noting *how many* dispute reasons exist, as that could play a major role in the decision.

In [22]:
dispute_reason_df['DISPUTE_REASON_KEY'].nunique()

330

In [23]:
dispute_reason_df['DISPUTE_REASON_KEY'].sort_values()

327    -99
326    -98
83       1
84       2
85       3
      ... 
215    324
216    325
217    326
218    327
325    328
Name: DISPUTE_REASON_KEY, Length: 330, dtype: int64

There seem to be a total of 330 distinct dispute reasons. While a 330-category classification is likely not useful (and therefore we'd have to use the binary yes/no chargeback as the target variable instead), it's possible that we won't actually be using all of the categories. 

Notice that there are two negative values, -98 and -99. Let's take a closer look:

In [24]:
dispute_reason_df[dispute_reason_df['DISPUTE_REASON_KEY'].isin([-99,-98])]

,DISPUTE_REASON_KEY,DISPUTE_REASON_DESC,DISPUTE_GROUP_DESC,WORKFLOW_TYPE
326,-98,NaN,NaN,NaN
327,-99,NaN,NaN,NaN


As suspected, these are empty rows and can be removed from the dataset.

In [25]:
dispute_reason_df = dispute_reason_df[~dispute_reason_df['DISPUTE_REASON_KEY'].isin([-99,-98])] # ~ denotes "not"

This indicates there may be more missing values. Since we know the Dispute Reason Key won't be missing, we can use the pandas' dropna command as is to remove all empty samples:

In [26]:
dispute_reason_df.dropna()

,DISPUTE_REASON_KEY,DISPUTE_REASON_DESC,DISPUTE_GROUP_DESC,WORKFLOW_TYPE
0,126,Fraud Liability Shift - Lost or Stolen,Unknown Dispute Group,Unknown Workflow Type
1,127,Cancelled recurring transaction,Old Visa Chargeback Reason Code,Old Visa Chargeback Reason Code
2,128,Non-matching account number,Old Visa Chargeback Reason Code,Old Visa Chargeback Reason Code
3,129,Paid by other means (U.S. only),Old Visa Chargeback Reason Code,Old Visa Chargeback Reason Code
4,130,Request for Support Illegible/Incomplete,Unknown Dispute Group,Unknown Workflow Type
...,...,...,...,...
319,309,Local Regulatory/Legal Dispute,Unknown Dispute Group,Unknown Workflow Type
320,310,Fraud Liability Shift - Counterfeit,Unknown Dispute Group,Unknown Workflow Type
321,311,Not as described or defective merchandise,Old Visa Chargeback Reason Code,Old Visa Chargeback Reason Code
322,312,No authorization,Old Visa Chargeback Reason Code,Old Visa Chargeback Reason Code


We see that five more rows have been removed. Now that we've seen the various possible reasons for chargeback, let's see how the transactions were actually labelled.

### Labels for Transactions

These are the transaction labels - the target variable. The table simply consists of the transaction key and the dispute reason key. If the dispute reason is zero, no chargeback occured. We load the two (zipped) files and can use a simple concatenation to put them together into a single table:

In [27]:
l1_df = pd.read_csv("data/labels_01-15-05-2020.csv.bz2", compression = 'bz2')
l2_df = pd.read_csv("data/labels_16-31-05-2020.csv.bz2", compression = 'bz2')
labels = [l1_df, l2_df]
labels_df = pd.concat(labels)
labels_df

,UNIQUE_TRX_KEY,DISPUTE_REASON_KEY
0,939083549,138
1,939091974,276
2,939083999,276
3,939099296,196
4,939094745,237
...,...,...
3048382,926364534,0
3048383,927421642,0
3048384,929440069,0
3048385,933972724,0


Let's observe this data. How many unique chargeback reasons are there?

In [28]:
labels_df.nunique()

UNIQUE_TRX_KEY        6023968
DISPUTE_REASON_KEY         40
dtype: int64

That's much fewer than the 323 possible categories we observed earlier. With 40 categories it might be reasonable to try to predict the chargeback category instead of having the model try and classify the binary yes/no chargeback question. Think about it this way: if the model is better at predicting the actual categories (with "no chargeback" being one of the 40) then we can simply sum up the categories in the end as "yes chargeback" with potentially better results.

There's no guarantee that'll work - certainly when we're working with what we expect to be highly imbalanced data - but it could be worth considering later. Regardless, we add a binary yes/no chargeback label as well, as an alternative target variable which we can choose to use later.

In [29]:
labels_df['CHARGEBACK'] = labels_df['DISPUTE_REASON_KEY'] != 0

Let's check for duplicates as well. 

### Transactions: the main table

Loading main features table (31 files, one per day of month) using file concatenation:

In [30]:
trx = []
for day in [str(x+1).zfill(2) for x in range(31)]:  # '01', '02' ... '31'
    file_name = 'data/result_' + day + '-05-2020.csv.bz2'
    trx.append(pd.read_csv(file_name, compression='bz2'))
trx_df = pd.concat(trx)

In [31]:
trx_df.shape

(6023968, 14)

We now can join all of the supplementary data tables to create a full dataset with multiple features relating to AVS and CVV authentication, the region(s) related to the transaction, the type of merchant, and the many chargeback reasons.  

In [32]:
trx_df = trx_df.set_index('UNIQUE_TRX_KEY').join(labels_df.drop_duplicates().set_index('UNIQUE_TRX_KEY')).reset_index()
trx_df = trx_df.set_index('MCC_KEY').join(mcc_df.set_index('MCC_KEY'))
trx_df = trx_df.set_index('AVS_RESULT_KEY').join(avs_df.set_index('AVS_RESPONSE_KEY'))
trx_df = trx_df.set_index('CVV_RESULT_KEY').join(cvv_df.set_index('CVV_RESPONSE_KEY'), )
trx_df = trx_df.set_index('TRX_REGION_KEY').join(region_df.set_index('TRX_REGION_KEY'))
trx_df = trx_df.set_index('DISPUTE_REASON_KEY').join(dispute_reason_df.set_index('DISPUTE_REASON_KEY'))
# drop duplicates?

In [33]:
trx_df

,UNIQUE_TRX_KEY,REQUEST_TIME_UTC,TRX_CURRENCY,TRX_AMOUNT_ORG,TRX_AMOUNT_USD,COUNTRY_BY_IP,MID,THREEDS,COUNTRY_BY_BIN,CARD_SCHEME_DESC,...,MERCHANT_COUNTRY_NUMERIC_ID,MERCHANT_COUNTRY_DESC,CARD_SCHEME_ID,CARD_SCHEME_TRX_REGION,CRX_TRX_REGION,MD5_KEY,ETL_SOURCE_PRIORITY,DISPUTE_REASON_DESC,DISPUTE_GROUP_DESC,WORKFLOW_TYPE
DISPUTE_REASON_KEY,,,,,,,,,,,,,,,,,,,,,
0,915797123,2020-05-04 15:40:48.000,USD,0.99,0.990000,USA,cce9fcd033c9ae9f42e2923e4f15af7be6540370cce1ae...,0,ZAR,Mastercard,...,-1,Unknown country,-1,Unknown Card Scheme Region,Unknown Crx Region,NaN,NaN,NaN,NaN,NaN
0,926961956,2020-05-21 12:49:31.000,USD,17.95,17.950000,COD,cce9fcd033c9ae9f42e2923e4f15af7be6540370cce1ae...,0,ZAR,Mastercard,...,-1,Unknown country,-1,Unknown Card Scheme Region,Unknown Crx Region,NaN,NaN,NaN,NaN,NaN
0,927577660,2020-05-22 13:11:33.000,USD,17.95,17.950000,COD,cce9fcd033c9ae9f42e2923e4f15af7be6540370cce1ae...,0,ZAR,Mastercard,...,-1,Unknown country,-1,Unknown Card Scheme Region,Unknown Crx Region,NaN,NaN,NaN,NaN,NaN
0,927873955,2020-05-22 13:06:49.000,USD,15.95,15.950000,COD,cce9fcd033c9ae9f42e2923e4f15af7be6540370cce1ae...,0,ZAR,Mastercard,...,-1,Unknown country,-1,Unknown Card Scheme Region,Unknown Crx Region,NaN,NaN,NaN,NaN,NaN
0,931041793,2020-05-27 21:15:27.000,USD,15.95,15.950000,UGA,cce9fcd033c9ae9f42e2923e4f15af7be6540370cce1ae...,0,ZAR,Mastercard,...,-1,Unknown country,-1,Unknown Card Scheme Region,Unknown Crx Region,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,930962472,2020-05-27 16:12:16.000,HKD,1000.00,128.996290,-1,aae0d223704d434e20da7c4f655c9a0d6d1058cc5280be...,0,HKG,Visa,...,826,United Kingdom,Visa,Interregional,Inter,456219565EDF917386F5EABFBEE9D09C,NaN,VCR Credit Not Processed,Visa Consumer Disputes - Collaboration,Visa Consumer Disputes - Collaboration
305,931139492,2020-05-27 07:04:58.000,HKD,1000.00,128.996290,-1,aae0d223704d434e20da7c4f655c9a0d6d1058cc5280be...,0,HKG,Visa,...,826,United Kingdom,Visa,Interregional,Inter,456219565EDF917386F5EABFBEE9D09C,NaN,VCR Credit Not Processed,Visa Consumer Disputes - Collaboration,Visa Consumer Disputes - Collaboration
305,931287381,2020-05-27 20:06:51.000,HKD,1000.00,128.996290,-1,aae0d223704d434e20da7c4f655c9a0d6d1058cc5280be...,0,HKG,Visa,...,826,United Kingdom,Visa,Interregional,Inter,456219565EDF917386F5EABFBEE9D09C,NaN,VCR Credit Not Processed,Visa Consumer Disputes - Collaboration,Visa Consumer Disputes - Collaboration


In [34]:
# why 

# drop duplicates?

In [35]:
trx_df.dtypes

UNIQUE_TRX_KEY                   int64
REQUEST_TIME_UTC                object
TRX_CURRENCY                    object
TRX_AMOUNT_ORG                 float64
TRX_AMOUNT_USD                 float64
COUNTRY_BY_IP                   object
MID                             object
THREEDS                          int64
COUNTRY_BY_BIN                  object
CARD_SCHEME_DESC                object
CHARGEBACK                        bool
MCC_ID                           int64
IS_VISA_HIGH_RISK                int64
IS_MC_HIGH_RISK                  int64
MCC_GROUP                       object
AVS_RESPONSE_ID                 object
AVS_RESPONSE_DESC               object
AVS_ADDRESS_MATCH                int64
AVS_ZIP_YES_MATCH                int64
AVS_ZIP_NO_MATCH                 int64
CVV_RESPONSE_ID                 object
CVV_RESPONSE_DESC               object
CVV_YES_MATCH                    int64
CVV_NO_MATCH                     int64
BIN_COUNTRY_ALPHA_3_ID          object
BIN_COUNTRY_ALPHA_2_ID   

After merging the tables using the keys, we can drop the irrelevant features:

In [36]:
trx_df.drop(["MD5_KEY", "ETL_SOURCE_PRIORITY", "DISPUTE_REASON_DESC", "DISPUTE_GROUP_DESC", "WORKFLOW_TYPE"],
            inplace=True, axis=1)  # mcc id?

Some more cleanup: changing the REQUEST_TIME_UTC into datetime format, and getting rid of more columns which we deem to be irrelevant to the prediction of chargeback:

In [37]:
trx_df['REQUEST_TIME_UTC'] = pd.to_datetime(trx_df['REQUEST_TIME_UTC'])

(TODO: change labels below)

In [39]:
labels_gb = trx_df.groupby(by='UNIQUE_TRX_KEY').count().sort_values(by='DISPUTE_REASON_KEY', ascending=False).reset_index()
labels_gb.rename(columns= {'DISPUTE_REASON_KEY': 'COUNT'}, inplace=True)
labels_gb[labels_gb['COUNT'] > 1]

KeyError: 'DISPUTE_REASON_KEY'

Notice that we seem to have a few duplicates. Let's check a few of these entries:

In [ ]:
labels_df[labels_df['UNIQUE_TRX_KEY'] == 925247439]

In this case all chargeback reasons are identical, so it's likely just an error in the data and we can remove the duplicates. However...

In [ ]:
labels_df[labels_df['UNIQUE_TRX_KEY'] == 914863078]

Here there seem to be two reasons for chargeback, as well as some duplicates. We have multiple choices how to deal with this imperfect data: we can treat the two rows as two separate transactions or decide to arbitrarily eliminate one of the chargeback reasons. Since there a likely no more than a few thousand of these cases (out of many millions) this likely won't make a difference and we'll stick with the first option, but it may be worth looking into this if we end up trying 40-category classification.

In [ ]:
#data exploration

### Imbalanced Data and a discussion about metrics

One of the most important questions we must ask in a classification problem is whether or not the dataset is *balanced*. An unbalanced dataset could lead to poor results when training the dataset, since it's easy to reach the prediction that all transaction be predicted as OK (with a misleadingly high accuracy score!).

For this problem we *expect* the data to be highly imbalanced, since most transaction are indeed OK. Let's see what our data tell us: 

In [ ]:
binary_labels_gb = binary_labels_df.groupby(by='CHARGEBACK')['UNIQUE_TRX_KEY'].count().reset_index()
binary_labels_gb['PERC'] = 100 * binary_labels_gb['UNIQUE_TRX_KEY']  / binary_labels_gb['UNIQUE_TRX_KEY'].sum()
binary_labels_gb.rename(columns={'UNIQUE_TRX_KEY': 'COUNT'}, inplace=True)
binary_labels_gb

Out of the entire dataset, only  of transactions are chargebacks! 

## Exploratory Data Analysis

Now that we've consolidated our data into one major table, it's time to examine our features in depth and, most importantly, to understand their relationship with the target variable.  

In [ ]:
res_df.info(verbose=True, null_counts=True)

In [ ]:
# changing data type can save lots of memory space, implement a few changes if necessary (and doesn't hurt model)

### Country

Let's see where the chargeback is happening:

In [ ]:
country_gb = res_labels_df.groupby(by = 'COUNTRY_BY_BIN').agg({"CHARGEBACK": 'sum', "MID": 'count'}).reset_index()
country_gb.rename(columns = {'MID': 'TOTAL_TRX'}, inplace=True)
country_gb['PERC'] = country_gb['CHARGEBACK']/country_gb['TOTAL_TRX']
country_gb.sort_values(by='CHARGEBACK', ascending=False)

We notice the following:
1. 
2.

In [ ]:
country_gb.sort_values(by='TOTAL_TRX', ascending=False)

There are countries with very few transactions; from the entire dataset, there is only one transaction which occured in Libya etc.

### Card Type

In [ ]:
card_gb = res_labels_df.groupby(by = 'CARD_SCHEME_DESC').agg({"CHARGEBACK": 'sum', "MID": 'count'}).reset_index()
card_gb.rename(columns = {'MID': 'TOTAL_TRX'}, inplace=True)
card_gb['PERC'] = card_gb['CHARGEBACK']/card_gb['TOTAL_TRX']
card_gb.sort_values(by='CHARGEBACK', ascending=False)

### Merchants

In [ ]:
mcc_gb = mcc_join.groupby(by = 'MCC_GROUP').agg({"CHARGEBACK": 'sum', "MID": 'count'}).reset_index()
mcc_gb.rename(columns = {'MID': 'TOTAL_TRX'}, inplace=True)
mcc_gb['PERC'] = mcc_gb['CHARGEBACK']/mcc_gb['TOTAL_TRX']
mcc_gb.sort_values(by='CHARGEBACK', ascending=False)

In [ ]:
mcc_gb.sort_values(by='PERC', ascending=False)

In [ ]:
mid_check = res_labels_df.groupby(by = 'MID').agg({"CHARGEBACK": 'sum', "UNIQUE_TRX_KEY": 'count'}).reset_index()
mid_check.rename(columns = {'UNIQUE_TRX_KEY': 'TOTAL_TRX'}, inplace=True)
mid_check['PERC'] = mid_check['CHARGEBACK']/mid_check['TOTAL_TRX']
mid_check.sort_values(by='CHARGEBACK', ascending=False)

In [ ]:
import matplotlib.pyplot as pls 
mid_copy = mid_check.sort_values(by='CHARGEBACK', ascending=False).iloc[0:600, :]
mid_copy.plot(x='MID', y='CHARGEBACK', kind='bar') 
plt.show()

We now check the IS_VISA_HIGH_RISK feature - does a high-risk Visa card lead to a higher percentage of chargebacks?

In [ ]:
mcc_visa_risk_gb = mcc_join.groupby(by = 'IS_VISA_HIGH_RISK').agg({"CHARGEBACK": 'sum', "MID": 'count'}).reset_index()
mcc_visa_risk_gb.rename(columns = {'MID': 'TOTAL_TRX'}, inplace=True)
mcc_visa_risk_gb['PERC'] = mcc_visa_risk_gb['CHARGEBACK']/mcc_visa_risk_gb['TOTAL_TRX']
mcc_visa_risk_gb.sort_values(by='CHARGEBACK', ascending=False)

That seems quite counterintuitive. Perhaps it's because...

### Currency

### Time of Day

## Further Modeling

### Separating time

In [ ]:
mod_df['DATE'] =  pd.DatetimeIndex(mod_df['REQUEST_TIME_UTC']).day
mod_df['DAY'] = pd.DatetimeIndex(mod_df['REQUEST_TIME_UTC']).dayofweek 
mod_df['HOUR'] =  pd.DatetimeIndex(mod_df['REQUEST_TIME_UTC']).hour

mod_df.drop("REQUEST_TIME_UTC", inplace=True, axis=1)
mod_df.head()

### Defining categorical features numerically

In [ ]:
# Visa (0) vs Mastercard (1)
mod_df.loc[(mod_df.CARD_SCHEME_DESC == 'Mastercard'),'CARD_SCHEME_DESC']= 1
mod_df.loc[(mod_df.CARD_SCHEME_DESC == 'Visa'),'CARD_SCHEME_DESC']= 0

# Assigning numbers to each currency category
mod_df["TRX_CURRENCY"] = mod_df["TRX_CURRENCY"].astype('category')
mod_df["CURRENCY"] = mod_df["TRX_CURRENCY"].cat.codes
mod_df.drop("TRX_CURRENCY", inplace=True, axis=1)



mod_df.head()

### One-hot Encoding for Categorical Features

In [ ]:
curr_enc = pd.get_dummies(mod_res.TRX_CURRENCY, prefix='Currency')
merc_enc = pd.get_dummies(mod_res.MCC_GROUP, prefix='Merch')
card_enc = pd.get_dummies(mod_res.CARD_SCHEME_DESC, prefix='Card')
day_enc = pd.get_dummies(mod_res.DAY, prefix='Day')

mod_res.drop("TRX_CURRENCY",  inplace=True, axis=1)
mod_res.drop("MCC_GROUP",  inplace=True, axis=1)
mod_res.drop("CARD_SCHEME_DESC",  inplace=True, axis=1)
mod_res.drop("DAY",  inplace=True, axis=1)

mod_res = mod_res.join(curr_enc)
mod_res = mod_res.join(merc_enc)
mod_res = mod_res.join(card_enc)
mod_res = mod_res.join(day_enc)

#delete clutter variables
del curr_enc
del merc_enc
del card_enc
del day_enc

mod_res.info()

In [ ]:
mod_df.set_index("UNIQUE_TRX_KEY")

### Rearranging the columns (fix, naturally)

In [ ]:
cols = mod_df.columns.tolist()
cols = cols[0:8] + cols[9:] + cols[8:9]

mod_df = mod_df[cols]
mod_df.head()

## Classification Modeling

Overview of classification techniques and deliberations

### Calculation of Profit

The following function calculates the profit based on the predictions (denoted "y_hat") compared to the actual result.

In [ ]:
def calculate_profit(y_actual, y_hat, amount):
    profit = 0
    for i in range(len(y_hat)): 
        if y_hat[i] == 1 and y_actual[i] == 0: # FP
           profit -= 2*amount[i]
        if y_actual[i] == y_hat[i] == 0: # TN
           profit += 0.15*amount[i]
        if y_hat[i] == 0 and y_actual[i] == 1: # FN
           profit -= 2.3*amount[i]
    return (profit)

However, presenting results in terms of absolute profit doesn't allow for a fair comparison between datasets with a different number of transactions; naturally if there are more samples a higher profit can be achieved, even if the model is substantially poorer.

To overcome this issue, we must **normalize** the profit metric. In our opinion, the best way to do this is to examine the maximum possible profit one can realize for the samples offered (i.e. the model is perfect) and divide the actual profit by this number. This results in a number between 0 and 1. The baseline model is normalized in a similar manner, thus allowing for a good comparison between it and our models.

Let us define a similar function which returns the maximum possible profit, when no error (false positive or true negative) was made:

In [ ]:
def maxvalid_profit(y_actual, amount):
    profit = 0
    for i in range(len(y_actual)): 
        if y_actual[i] == 0: # TN
           profit += 0.15*amount[i]
    return (profit)

### Specifying the Target Variable

In [ ]:
# Splitting the dataset into features (X) and target variable (Y).
X = mod_res.values[:, :-1]
Y = mod_res.values[:,-1]
Y=Y.astype('int')

### Splitting data into training-validation-test

In [ ]:
from sklearn.model_selection import train_test_split
x_trainAndVal, x_test, y_trainAndVal, y_test = train_test_split(X, Y, test_size=0.75, random_state = 123)
x_train, x_validation, y_train, y_validation = train_test_split(x_trainAndVal, y_trainAndVal, test_size=0.30, random_state = 123)
x_train_base, x_train_op, y_train_base, y_train_op = train_test_split(x_train, y_train, test_size=0.10, random_state = 123)

### Downsampling

As mentioned above, due to the highly imbalanced nature of the dataset, some technique must be used to avoid overfitting and improve the metrics. Since we have a very large number of samples, it makes sense to **downsample** (= remove a certain percentage of the majority (non-chargeback) samples) our data which can both improve performance and allow for faster modeling. 

Other methods for dealing with imbalanced data include ......... 

In [ ]:
DownsamplingFlag = True

if DownsamplingFlag:
  from sklearn.utils import resample

  mod_res_train_base = np.c_[x_train_base, y_train_base]
  mod_res_majority = mod_res_train_base[mod_res_train_base[:,-1]==0]
  mod_res_minority = mod_res_train_base[mod_res_train_base[:,-1]==1]
 
  # Downsample majority class
  mod_res_majority_downsampled = resample(mod_res_majority, 
                                   replace=False,    # sample without replacement
                                   n_samples=int(0.2*len(y_train_base)), #make data ~5%    
                                   random_state=23) # reproducible results. Tried on 5, 23,
 
  # Combine minority class with downsampled majority class
  mod_res = np.r_[mod_res_majority_downsampled, mod_res_minority]
 
  #delete clutter variables
  del mod_res_minority
  del mod_res_majority
  del mod_res_majority_downsampled

  x_train_base = mod_res[:, :-1]
  y_train_base = mod_res[:,-1]
  y_train_base = y_train_base.astype('int')

### Baseline Model

In machine learning, it is highly recommended to decide on a simple baseline model to which other models can be compared to. In this case, the simplest model happens to be the company's current policy: assume all transactions are OK (no chargebacks), which for the vast majority of cases is entirely correct! That being said, the company understand the long and short-term losses which the type II (false negative) errors accrue, and that's what we're trying to accomplish by building a better classification model.

Let's examine the companies profit (normalized by the maximum possible profit, as mentioned previously) if all transactions are predicted to be OK (no chargeback). 

### Logistic Regression

One of the simpler models worth trying out is logistic regression. In this model, 

In [ ]:
from sklearn.linear_model import LogisticRegression
lr_mdl = LogisticRegression(random_state=0).fit(x_train_base, y_train_base)

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt_mdl = DecisionTreeClassifier().fit(x_train_base, y_train_base)

### Random Forest

Random Forest is generally consider a highly robust and useful model, which can be further improved by hyperparameter tuning, as we shall see later.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_mdl = RandomForestClassifier(max_depth=40, random_state=123).fit(x_train_base, y_train_base)

### XGBoost

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
data_dmatrix = xgb.DMatrix(data=x_train_base,label=y_train_base)
xgb_mdl = xgb.XGBClassifier(learning_rate =0.1,n_estimators=300,max_depth=5,min_child_weight=1,gamma=0,subsample=0.8,
                          colsample_bytree=0.8,objective='binary:logistic',nthread=4,scale_pos_weight=1,seed=125)
                        .fit(x_train, y_train)

XGBoost allows us to examine feature importance, in order to potentially isolate features which are irrelevant to the target variable and may results in unnecessary overfitting:

In [ ]:
from matplotlib import pyplot
from xgboost import plot_importance
from xgboost import Booster
mdl.get_booster().feature_names = list(mod_res.columns)
plot_importance(mdl)

### Making the predictions

The following function takes the validation set and predicts the target variable for each sample, after which the predicted values can be compared to the actual values in order to compute the total profit (see the "calculate_profit" function defined earlier) normalized by the maximum profit (see the "max_valid_profit" function). The function returns the profit, the normalized profit, and the auxiliary F1 score; by defining this function we can test how the predictions fare for various classification threshold and thus optimize using the ideal threshold.  

In [ ]:
def predictions (x_validation, y_validation, threshold, mdl):

  y_pred_prob = mdl.predict_proba(x_validation)      
    # We use the predict_proba function to compute probability, instead of using the predict function directly. 
  y_pred = np.where(y_pred_prob[:,1] > threshold, 1, 0)    # If probabilty for CB larger than threshold, mark as CB.
  #sum(y_pred)/len(y_pred)     # % of transactions marked as risky for chargebacks.
  model_profit = calculate_profit(y_validation, y_pred, x_validation[:,2]) 
  normalized_profit = model_profit / maxvalid_profit(y_validation, x_validation[:,2])
  tn, fp, fn, tp = confusion_matrix(y_pred,y_validation).ravel()
  print(tn, fp, fn, tp)
  precision = tp/(tp+fp)
  recall = tp/(tp+fn)
  F1 = 2 * (precision * recall) / (precision + recall)
  return [model_profit, normalized_profit, F1]